In [37]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [38]:
#DataSet Limpio con columnas no utiles segun analisis droppeadas.

oportunidades = pd.read_csv("Train_TP2_Datos_2020-2C.csv")
oportunidades['Account_Created_Date'] = pd.to_datetime(oportunidades['Account_Created_Date'])
oportunidades['Opportunity_Created_Date'] = pd.to_datetime(oportunidades['Opportunity_Created_Date'])
oportunidades['Quote_Expiry_Date'] = pd.to_datetime(oportunidades['Quote_Expiry_Date'])
oportunidades['Last_Modified_Date'] = pd.to_datetime(oportunidades['Last_Modified_Date'])
oportunidades['Planned_Delivery_Start_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_Start_Date'])
oportunidades['Planned_Delivery_End_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_End_Date'])
oportunidades_japon = (oportunidades.loc[oportunidades['Region'] == 'Japan'])
oportunidades = (oportunidades.loc[oportunidades['Region'] != 'Japan'])
#oportunidades_japon = oportunidades_japon.iloc[:,:3]
oportunidades_japon['Territory'] = oportunidades_japon['Territory'].replace({'None':'Japan'})
oportunidades = pd.concat([oportunidades, oportunidades_japon], axis=0)
oportunidades['Region'] = oportunidades['Region'].replace({'Japan':'APAC', 'Middle East':'EMEA'})
oportunidades[oportunidades.select_dtypes(['object']).columns] = oportunidades.select_dtypes(['object']).apply(lambda x: x.astype('category'))
oportunidades = oportunidades.loc[oportunidades['Territory'] != 'None']
oportunidades['Conversion_Moneda'] = oportunidades['ASP'] / oportunidades['ASP_(converted)']
oportunidades['Total_Amount'] = oportunidades['Total_Amount'] / oportunidades['Conversion_Moneda']
oportunidades['Total_Taxable_Amount'] = oportunidades['Total_Taxable_Amount'] / oportunidades['Conversion_Moneda']
oportunidades = oportunidades[(oportunidades['Stage'] == 'Closed Won') | (oportunidades['Stage'] == 'Closed Lost')]
oportunidades['Stage'] = oportunidades['Stage'].replace({'Closed Won':1, 'Closed Lost':0})
oportunidades = oportunidades.drop(['Pricing, Delivery_Terms_Quote_Appr'\
    ,'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval'\
        , 'Bureaucratic_Code_0_Approved','Submitted_for_Approval', \
    'Billing_Country','Account_Created_Date', 'Opportunity_Name','Delivery_Year',\
   'Actual_Delivery_Date','Prod_Category_A','Product_Category_B','Delivery_Quarter','Month',\
   'Planned_Delivery_End_Date','Planned_Delivery_Start_Date','ASP_(converted)_Currency','ASP_Currency',\
   'ASP','ASP_(converted)', 'Sales_Contract_No', 'Bureaucratic_Code', 'Opportunity_Created_Date', 'Brand',\
   'Product_Type','Size','Price','Currency','Last_Activity','Quote_Expiry_Date','Last_Modified_Date','Last_Modified_By','ID',\
   'Delivery_Terms','Conversion_Moneda', 'Total_Taxable_Amount_Currency', 'Quote_Type'], axis = 'columns')
oportunidades = oportunidades.dropna()
oportunidades

,Region,Territory,Source,Account_Name,Opportunity_ID,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Product_Family,Product_Name,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount,Stage
2,Americas,NW America,Source_7,Account_Name_1794,2,Person_Name_64,Person_Name_8,Account_Type_5,Opportunity_Type_1,Product_Family_81,Product_Name_91,0,USD,8.386560e+04,8.386560e+04,1
3,Americas,NW America,Source_11,Account_Name_1201,3,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Product_Family_209,Product_Name_432,14,USD,7.421882e+06,7.421882e+06,0
4,Americas,NW America,Source_11,Account_Name_1201,4,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Product_Family_209,Product_Name_432,25,USD,1.335719e+07,1.335719e+07,0
5,Americas,NW America,Source_11,Account_Name_1201,5,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Product_Family_209,Product_Name_432,28,USD,1.483828e+07,1.483828e+07,0
6,Americas,NW America,Source_11,Account_Name_1201,6,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Product_Family_164,Product_Name_308,7,USD,2.659495e+06,2.659495e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16922,APAC,Japan,Source_7,Account_Name_1294,12785,Person_Name_66,Person_Name_66,Account_Type_2,Opportunity_Type_1,Product_Family_243,Product_Name_508,0,USD,1.320000e+04,6.422401e+06,1
16923,APAC,Japan,Source_3,Account_Name_1074,12786,Person_Name_61,Person_Name_66,Account_Type_0,Opportunity_Type_1,Product_Family_77,Product_Name_100,0,JPY,1.096295e+05,1.096295e+05,1
16924,APAC,Japan,Source_3,Account_Name_1074,12787,Person_Name_61,Person_Name_66,Account_Type_0,Opportunity_Type_1,Product_Family_73,Product_Name_85,0,JPY,1.075213e+05,1.075213e+05,1
16925,APAC,Japan,Source_11,Account_Name_1522,12788,Person_Name_50,Person_Name_30,Account_Type_0,Opportunity_Type_10,Product_Family_77,Product_Name_100,0,JPY,1.576650e+05,1.576650e+05,1


In [39]:
# Encodeos necesarios:
#----------------------------------------------------------------------------------------------------
# Total_Amount_Currency(AUD y JPY)
# Region (APAC)
# Territorio (Mean Stage)
# Dif_Amount(TAm-TTAm)
# TRF > 0
# Product_Family(En base a ranking o mean de Stage)
# Product_Name(En base a ranking o mean de Stage)
# Total Amount y Total Tax hay que dropearlos despues del Dif_Amount
# Account Name (Ranking o Mean Stage)
# Opportunity Owner (Ranking o Mean Stage)
# Account Owner (Ranking o Mean Stage)
# Account Type (Ranking o Mean Stage)
# Opportunity Type (Ranking o Mean Stage)
# Source (Decidir el encodeo)
#---------------------------------------------------------------------------------------------------

In [40]:
# Total_Amount_Currency(AUD y JPY)
# Region (APAC)

oportunidades['Currency_Es_JPY_O_AUD'] = ((oportunidades['Total_Amount_Currency'] == 'AUD') | \
                                  (oportunidades['Total_Amount_Currency'] == 'JPY')).replace({True:1,False:0})
oportunidades['Region_Es_APAC'] = (oportunidades['Region'] == 'APAC').replace({True:1,False:0})
oportunidades = oportunidades.drop(columns=['Region','Total_Amount_Currency'])

In [41]:
# Territorio (Mean Stage)

gby_terr = oportunidades.groupby('Territory').agg({'Stage':'mean'})
gby_terr['Stage'] = gby_terr['Stage'].replace({np.nan:(gby_terr['Stage'].mean())})
terrs_sorted = gby_terr.sort_values(by='Stage')
terrs_sorted = terrs_sorted.reset_index()
terrs_sorted.columns = ['Territory', 'Encoding_Territory']
oportunidades = oportunidades.merge(terrs_sorted, on='Territory')
oportunidades = oportunidades.drop(columns=['Territory'])

In [42]:
# Dif_Amount(TAm-TTAm)

amounts = oportunidades.groupby('Opportunity_ID').agg({'Total_Amount':'sum','Total_Taxable_Amount':'mean'})
amounts['Diferencia_Amounts'] = amounts['Total_Amount']-amounts['Total_Taxable_Amount']
amounts = amounts.drop(columns = ['Total_Amount','Total_Taxable_Amount'])
amounts = amounts.reset_index()
oportunidades = oportunidades.merge(amounts, on='Opportunity_ID')
oportunidades = oportunidades.drop(columns = ['Total_Amount','Total_Taxable_Amount'])


In [43]:
# TRF > 0

trf = oportunidades.groupby('Opportunity_ID').agg({'TRF':'sum'})
trf['TRF_Mayor_A_0'] = trf['TRF']>0
trf = trf.drop(columns=['TRF'])
trf = trf.reset_index()
oportunidades = oportunidades.merge(trf, on='Opportunity_ID')
oportunidades = oportunidades.drop(columns=['TRF'])
oportunidades['TRF_Mayor_A_0'] = oportunidades['TRF_Mayor_A_0'].replace({True:1,False:0})

In [44]:
# Product_Family(Mean Stage)

gby_prod_fam = oportunidades.groupby('Product_Family').agg({'Stage':'mean'})
gby_prod_fam['Stage'] = gby_prod_fam['Stage'].replace({np.nan:(gby_prod_fam['Stage'].mean())})
p_fam_sorted = gby_prod_fam.sort_values(by='Stage')
p_fam_sorted = p_fam_sorted.reset_index()
p_fam_sorted.columns = ['Product_Family', 'Encoding_Prod_Family']
oportunidades = oportunidades.merge(p_fam_sorted, on='Product_Family')
oportunidades = oportunidades.drop(columns=['Product_Family'])

In [45]:
# Product_Name(En base a ranking)

gby_prod_name = oportunidades.groupby('Product_Name').agg({'Stage':'mean'})
gby_prod_name['Stage'] = gby_prod_name['Stage'].replace({np.nan:(gby_prod_name['Stage'].mean())})
p_name_sorted = gby_prod_name.sort_values(by='Stage')
p_name_sorted = p_name_sorted.reset_index()
p_name_sorted = p_name_sorted.reset_index()
p_name_sorted = p_name_sorted.drop(columns=['Stage'])
p_name_sorted.columns = ['Encoding_Prod_Name', 'Product_Name']
oportunidades = oportunidades.merge(p_name_sorted, on='Product_Name')
oportunidades = oportunidades.drop(columns=['Product_Name'])

In [46]:
# Account Name (Mean Stage)
gby_acc_name = oportunidades.groupby('Account_Name').agg({'Stage':'mean'})
gby_acc_name['Stage'] = gby_acc_name['Stage'].replace({np.nan:(gby_acc_name['Stage'].mean())})
acc_name_sorted = gby_acc_name.sort_values(by='Stage')
acc_name_sorted = acc_name_sorted.reset_index()
acc_name_sorted.columns = ['Account_Name', 'Encoding_Account_Name']
oportunidades = oportunidades.merge(acc_name_sorted, on='Account_Name')
oportunidades = oportunidades.drop(columns=['Account_Name'])

In [47]:
# Opportunity Owner (Ranking)

gby_op_ow = oportunidades.groupby('Opportunity_Owner').agg({'Stage':'mean'})
gby_op_ow['Stage'] = gby_op_ow['Stage'].replace({np.nan:(gby_op_ow['Stage'].mean())})
op_ow_sorted = gby_op_ow.sort_values(by='Stage')
op_ow_sorted = op_ow_sorted.reset_index()
op_ow_sorted = op_ow_sorted.reset_index()
op_ow_sorted = op_ow_sorted.drop(columns=['Stage'])
op_ow_sorted.columns = ['Encoding_Opportunity_Owner', 'Opportunity_Owner']
oportunidades = oportunidades.merge(op_ow_sorted, on='Opportunity_Owner')
oportunidades = oportunidades.drop(columns=['Opportunity_Owner'])

In [48]:
# Account Owner (Mean Stage)

gby_acc_ow = oportunidades.groupby('Account_Owner').agg({'Stage':'mean'})
gby_acc_ow['Stage'] = gby_acc_ow['Stage'].replace({np.nan:(gby_acc_ow['Stage'].mean())})
acc_ow_sorted = gby_acc_ow.sort_values(by='Stage')
acc_ow_sorted = acc_ow_sorted.reset_index()
acc_ow_sorted.columns = ['Account_Owner', 'Encoding_Account_Owner']
oportunidades = oportunidades.merge(acc_ow_sorted, on='Account_Owner')
oportunidades = oportunidades.drop(columns=['Account_Owner'])

In [49]:
# Account Type (Ranking)

gby_acc_type = oportunidades.groupby('Account_Type').agg({'Stage':'mean'})
gby_acc_type['Stage'] = gby_acc_type['Stage'].replace({np.nan:(gby_acc_type['Stage'].mean())})
acc_type_sorted = gby_acc_type.sort_values(by='Stage')
acc_type_sorted = acc_type_sorted.reset_index()
acc_type_sorted = acc_type_sorted.reset_index()
acc_type_sorted = acc_type_sorted.drop(columns=['Stage'])
acc_type_sorted.columns = ['Encoding_Account_Type', 'Account_Type']
oportunidades = oportunidades.merge(acc_type_sorted, on='Account_Type')
oportunidades = oportunidades.drop(columns=['Account_Type'])

In [50]:
# Opportunity Type (Mean Stage)

gby_op_type = oportunidades.groupby('Opportunity_Type').agg({'Stage':'mean'})
gby_op_type['Stage'] = gby_op_type['Stage'].replace({np.nan:(gby_op_type['Stage'].mean())})
op_type_sorted = gby_op_type.sort_values(by='Stage')
op_type_sorted = op_type_sorted.reset_index()
op_type_sorted.columns = ['Opportunity_Type', 'Encoding_Opportunity_Type']
oportunidades = oportunidades.merge(op_type_sorted, on='Opportunity_Type')
oportunidades = oportunidades.drop(columns=['Opportunity_Type'])

In [51]:
# Source (Decidir el encodeo)

In [52]:
oportunidades

,Source,Opportunity_ID,Stage,Currency_Es_JPY_O_AUD,Region_Es_APAC,Encoding_Territory,Diferencia_Amounts,TRF_Mayor_A_0,Encoding_Prod_Family,Encoding_Prod_Name,Encoding_Account_Name,Encoding_Opportunity_Owner,Encoding_Account_Owner,Encoding_Account_Type,Encoding_Opportunity_Type
0,Source_7,2,1,0,0,0.631246,0.0,0,0.833333,379,0.826667,39,0.675481,3,0.545089
1,Source_7,1315,1,0,0,0.631246,0.0,0,0.325779,301,0.826667,39,0.675481,3,0.545089
2,Source_7,1316,1,0,0,0.631246,0.0,0,0.325779,301,0.826667,39,0.675481,3,0.545089
3,Source_7,2889,1,0,0,0.631246,0.0,0,0.325779,301,0.826667,39,0.675481,3,0.545089
4,Source_7,7,1,0,0,0.631246,0.0,0,0.636364,327,0.826667,39,0.675481,3,0.545089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12637,None,3517,0,0,1,0.272727,0.0,1,0.612613,147,0.125000,14,0.807074,5,0.333333
12638,None,3518,0,0,1,0.272727,0.0,1,0.612613,147,0.125000,14,0.807074,5,0.333333
12639,Source_7,2782,1,0,1,0.684917,0.0,1,0.818182,388,0.882353,52,0.700980,5,0.333333
12640,Source_7,2783,1,0,1,0.684917,0.0,1,0.818182,388,0.882353,52,0.700980,5,0.333333


In [53]:
x_train = oportunidades.groupby('Opportunity_ID').agg({'Currency_Es_JPY_O_AUD':'mean', 'Region_Es_APAC':'mean',\
                                 'Encoding_Territory':'mean', 'Diferencia_Amounts':'mean', 'TRF_Mayor_A_0':'mean',\
                                 'Encoding_Prod_Family':'mean','Encoding_Prod_Name':'mean','Encoding_Account_Name':'mean',\
                                 'Encoding_Opportunity_Owner':'mean','Encoding_Account_Owner':'mean','Encoding_Account_Type':'mean',\
                                  'Encoding_Opportunity_Type':'mean'})
y_train = oportunidades.groupby('Opportunity_ID').agg({'Stage':'max'})
division_x_train, division_x_test, division_y_train, division_y_test = train_test_split(x_train, y_train, test_size = 0.3,\
                                                                                       random_state = 123)
x_train

,Currency_Es_JPY_O_AUD,Region_Es_APAC,Encoding_Territory,Diferencia_Amounts,TRF_Mayor_A_0,Encoding_Prod_Family,Encoding_Prod_Name,Encoding_Account_Name,Encoding_Opportunity_Owner,Encoding_Account_Owner,Encoding_Account_Type,Encoding_Opportunity_Type
Opportunity_ID,,,,,,,,,,,,
2,0,0,0.631246,0.00,0,0.833333,379.0,0.826667,39,0.675481,3,0.545089
3,0,0,0.631246,0.00,1,0.243767,99.0,0.000000,39,0.594979,3,0.222024
4,0,0,0.631246,0.00,1,0.243767,99.0,0.000000,39,0.594979,3,0.222024
5,0,0,0.631246,0.00,1,0.243767,99.0,0.000000,39,0.594979,3,0.222024
6,0,0,0.631246,0.00,1,0.325779,127.0,0.000000,39,0.594979,3,0.222024
...,...,...,...,...,...,...,...,...,...,...,...,...
12799,0,0,0.421053,0.00,1,0.502816,186.0,0.670213,43,0.672492,6,0.545089
12800,0,0,0.335252,0.00,1,0.318974,122.5,0.000000,43,0.672492,3,0.222024
12801,0,0,0.855172,0.00,0,0.579137,341.0,1.000000,43,0.672492,3,0.545089


In [54]:
model = GridSearchCV(xgb.XGBRegressor(objective ='binary:logistic', n_estimators = 450, eval_metric='logloss'), {
    'learning_rate' : np.arange(0.008, 0.022, 0.004),
    'colsample_bytree' : np.arange(0.4,0.9,0.1),
    'alpha' : [0.0001,0.001,0.01,0.1],
    'max_depth' : np.arange(4,10),
    'gamma' : np.arange(0,0.3,0.05),
    'subsample' : np.arange(0.5,1,0.1)
}, cv = 2, return_train_score = False, n_jobs = -1, verbose=10,scoring=neg_log_loss)

model.fit(division_x_train, division_y_train)

Fitting 2 folds for each of 14400 candidates, totalling 28800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   46.9s
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

[Parallel(n_jobs=-1)]: Done 9361 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 9498 tasks      | elapsed: 43.6min
[Parallel(n_jobs=-1)]: Done 9637 tasks      | elapsed: 44.3min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 44.9min
[Parallel(n_jobs=-1)]: Done 9917 tasks      | elapsed: 45.5min
[Parallel(n_jobs=-1)]: Done 10058 tasks      | elapsed: 46.1min
[Parallel(n_jobs=-1)]: Done 10201 tasks      | elapsed: 46.8min
[Parallel(n_jobs=-1)]: Done 10344 tasks      | elapsed: 47.4min
[Parallel(n_jobs=-1)]: Done 10489 tasks      | elapsed: 48.1min
[Parallel(n_jobs=-1)]: Done 10634 tasks      | elapsed: 48.7min
[Parallel(n_jobs=-1)]: Done 10781 tasks      | elapsed: 49.4min
[Parallel(n_jobs=-1)]: Done 10928 tasks      | elapsed: 50.1min
[Parallel(n_jobs=-1)]: Done 11077 tasks      | elapsed: 50.8min
[Parallel(n_jobs=-1)]: Done 11226 tasks      | elapsed: 51.5min
[Parallel(n_jobs=-1)]: Done 11377 tasks      | elapsed: 52.1min
[Parallel(n_jobs=-1)]: Done 11528 tasks      

GridSearchCV(cv=2,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    eval_metric='logloss', gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_...
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),
             n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                     

In [55]:
accuracy_score(division_y_test, model.predict(division_x_test).round())

0.8313030638612035

In [56]:
log_loss(division_y_test, model.predict(division_x_test))

0.3577076656708839

In [57]:
model.best_params_

{'alpha': 0.1,
 'colsample_bytree': 0.5,
 'gamma': 0.1,
 'learning_rate': 0.02,
 'max_depth': 6,
 'subsample': 0.8999999999999999}